In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!mkdir -p /content/datasets/pk_lot
!unzip -q -o /content/drive/MyDrive/PKLot.v2-640.yolov11.zip \
    -d /content/datasets/pk_lot


In [ ]:
import yaml

DATA_ROOT = '/content/datasets/pk_lot'
with open(f"{DATA_ROOT}/data.yaml") as f:
    cfg = yaml.safe_load(f)

classes = cfg['names']
print("Classes:", classes)


In [ ]:
import os
SPLITS = {
    'train': {'imgs': os.path.join(DATA_ROOT, 'train', 'images'),
              'lbls': os.path.join(DATA_ROOT, 'train', 'labels')},
    'valid':{'imgs': os.path.join(DATA_ROOT, 'valid', 'images'),
             'lbls': os.path.join(DATA_ROOT, 'valid', 'labels')},
    'test' :{'imgs': os.path.join(DATA_ROOT, 'test', 'images'),
             'lbls': os.path.join(DATA_ROOT, 'test', 'labels')},
}

for split, p in SPLITS.items():
    print(f"{split:5}: {len(os.listdir(p['imgs']))} images, {len(os.listdir(p['lbls']))} labels")


In [ ]:
import glob
from PIL import Image

one = glob.glob(f"{SPLITS['train']['imgs']}/*.*")[0]
w,h = Image.open(one).size
imgsz = max(w, h)
print(f"Using imgsz = {w}×{h}")


In [ ]:
!pip install -q rfdetr==1.1.0 supervision pyyaml

In [ ]:
import json, glob, os
from PIL import Image

def convert_yolo_to_coco(split):
    img_dir = SPLITS[split]['imgs']
    lbl_dir = SPLITS[split]['lbls']
    out_json = os.path.join(DATA_ROOT, split, '_annotations.coco.json')

    images, annotations = [], []
    ann_id = 0

    for img_id, img_path in enumerate(sorted(glob.glob(f"{img_dir}/*.*"))):
        fn = os.path.basename(img_path)
        w, h = Image.open(img_path).size

        images.append({
            'id': img_id,
            'file_name': f"images/{fn}",
            'width': w,
            'height': h
        })

        lbl_file = os.path.join(lbl_dir, fn.replace(os.path.splitext(fn)[1], '.txt'))
        if os.path.exists(lbl_file):
            for line in open(lbl_file):
                cls, xc, yc, bw, bh = line.split()
                cls = int(cls)
                xc, yc, bw, bh = map(float, (xc, yc, bw, bh))
                x = (xc - bw/2) * w
                y = (yc - bh/2) * h
                width  = bw * w
                height = bh * h
                annotations.append({
                    'id': ann_id,
                    'image_id': img_id,
                    'category_id': cls,
                    'bbox': [x, y, width, height],
                    'area': width * height,
                    'iscrowd': 0
                })
                ann_id += 1

    categories = [
        {'id': i, 'name': name, 'supercategory': 'none'}
        for i, name in enumerate(classes)
    ]

    coco = {'images': images, 'annotations': annotations, 'categories': categories}
    with open(out_json, 'w') as f:
        json.dump(coco, f)
    print(f"[{split}] {len(images)} images, {ann_id} annotations → {out_json}")

for split in SPLITS:
    convert_yolo_to_coco(split)


In [ ]:
import os
from rfdetr import RFDETRBase
import supervision as sv
from tqdm import tqdm
import matplotlib.pyplot as plt
from supervision.metrics import MeanAveragePrecision

RUN_DIR  = '/content/drive/MyDrive/rfdetr_pklot_run'
CKPT_DIR = os.path.join(RUN_DIR, 'checkpoint')
os.makedirs(CKPT_DIR, exist_ok=True)


In [ ]:
model = RFDETRBase()
model.train(
    dataset_dir        = DATA_ROOT,
    epochs             = 3,
    batch_size         = 8,
    grad_accum_steps   = 2,
    lr                 = 1e-4,
    imgsz              = imgsz,
    fp16               = True,
    dataloader_workers = 8
)


In [ ]:
from PIL import Image

Image.open("/content/output/metrics_plot.png")

In [ ]:
import torch, os, yaml

os.makedirs(CKPT_DIR, exist_ok=True)

torch.save(model, os.path.join(CKPT_DIR, "rf_detr_base_full.pt"))

with open(os.path.join(CKPT_DIR, "training_config.yaml"), "w") as f:
    yaml.safe_dump({
        "imgsz": imgsz,
        "batch_size": 8,
        "epochs": 5,
        "lr": 1e-4,
        "classes": classes
    }, f)

print("checkpoint saved to", CKPT_DIR)


In [ ]:
import os
import supervision as sv
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from supervision.metrics import MeanAveragePrecision

DATA_ROOT = '/content/datasets/pk_lot'
RUN_DIR   = '/content/drive/MyDrive/rfdetr_pklot_run'

In [ ]:
eval_model = model

ds = sv.DetectionDataset.from_coco(
    images_directory_path = os.path.join(DATA_ROOT, 'test'),
    annotations_path      = os.path.join(DATA_ROOT, 'test', '_annotations.coco.json'),
)
print("Test classes:", ds.classes)
print("Number of test images:", len(ds))

In [ ]:
preds, targets = [], []
for img_path, _, ann in tqdm(ds, desc="Inferencing"):
    img  = Image.open(img_path)
    dets = eval_model.predict(img, threshold=0.5)
    preds.append(dets)
    targets.append(ann)

In [ ]:
map_metric = MeanAveragePrecision()
res        = map_metric.update(preds, targets).compute()

res.plot()
plt.gcf().savefig(os.path.join(RUN_DIR,'mAP.png'), bbox_inches='tight')
plt.close()
print("mAP plot → mAP.png")

In [ ]:
cm = sv.ConfusionMatrix.from_detections(
    predictions = preds,
    targets     = targets,
    classes     = ds.classes
)
cm.plot()
plt.gcf().savefig(os.path.join(RUN_DIR,'confusion_matrix.png'), bbox_inches='tight')
plt.close()
print("Confusion matrix → confusion_matrix.png")

In [ ]:
from IPython.display import Image, display
import os

conf_mat_path = os.path.join(RUN_DIR, 'confusion_matrix.png')
display( Image(filename=conf_mat_path) )

In [ ]:
path, _, ann = ds[0]
img          = Image.open(path)
dets         = eval_model.predict(img, threshold=0.5)

ts = sv.calculate_optimal_text_scale(resolution_wh=img.size)
th = sv.calculate_optimal_line_thickness(resolution_wh=img.size)
ba = sv.BoxAnnotator(thickness=th)
la = sv.LabelAnnotator(
    text_color     = sv.Color.BLACK,
    text_scale     = ts,
    text_thickness = th,
    smart_position = True
)

gt_lbls = [f"{ds.classes[c]}"        for c in ann.class_id]
dt_lbls = [f"{ds.classes[c]} {p:.2f}" for c,p in zip(dets.class_id, dets.confidence)]

gt_img = ba.annotate(img.copy(), ann)
gt_img = la.annotate(gt_img, ann, gt_lbls)
dt_img = ba.annotate(img.copy(), dets)
dt_img = la.annotate(dt_img, dets, dt_lbls)

sv.plot_images_grid(
    images    = [gt_img, dt_img],
    grid_size = (1, 2),
    titles    = ["Ground-Truth", "Detection"]
)

In [ ]:
det_images = []
for i in range(min(9, len(ds))):
    p, _, _ = ds[i]
    im      = Image.open(p)
    d       = eval_model.predict(im, threshold=0.5)
    o       = ba.annotate(im.copy(), d)
    o       = la.annotate(o, d, [f"{ds.classes[c]} {conf:.2f}"
                                 for c,conf in zip(d.class_id, d.confidence)])
    det_images.append(o)

sv.plot_images_grid(images=det_images, grid_size=(3,3), size=(12,12))
plt.gcf().savefig(os.path.join(RUN_DIR, 'detections_grid.png'), bbox_inches='tight')
plt.close()
print("Detections grid → detections_grid.png")


In [ ]:
!cp -r /content/output /content/drive/MyDrive/rfdetr_pklot_run/output

In [ ]:
import torch, os

os.makedirs(CKPT_DIR, exist_ok=True)

torch.save(model.model, os.path.join(CKPT_DIR, "model_module.pth"))
print(" model_module.pth saved →", CKPT_DIR)

## References:
RF-DETR: https://github.com/roboflow/rfdetr  
Supervision: https://github.com/roboflow/supervision  
PyYAML: https://pyyaml.org/wiki/PyYAMLDocumentation  
PyTorch: https://pytorch.org/docs/  
tqdm: https://tqdm.github.io/  
Matplotlib: https://matplotlib.org/stable/users/